# Text Generation with Transformers Pipeline

## Preparation

Install python packages.

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.6 MB/s eta 0:00:00


In [2]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.7 MB/s eta 0:00:00


In [3]:
!nvidia-smi

Mon Jul 17 04:01:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Fix random number seed.

In [4]:
import random
import numpy as np
import torch

def set_seed(seed=0):
  # for Python
  random.seed(seed)

  # for NumPy
  np.random.seed(seed)

  # for PyTorch, CUDA and cuDNN
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [5]:
set_seed(seed=0)

Get device.

In [6]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  device_id = 0
else:
  device = torch.device('cpu')
  device_id = -1

print(device)
print(device_id)

cuda
0


## Transformers pipeline

Load pretrained model and tokenizer.
For example, we use OpenCALM-1B

Here, for example, we use OpenCALM-1B (https://huggingface.co/cyberagent/open-calm-1b), which was pre-trained on Japanese datasets by CyberAgent, Inc.

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

pretrained_model_name = 'cyberagent/open-calm-1b'

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name, torch_dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

Build text generation pipeline.

In [8]:
from transformers import pipeline

task='text-generation'

kwargs = {
    'max_new_tokens': 64,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.9,
    'repetition_penalty': 1.05,
    'pad_token_id': tokenizer.pad_token_id
}

generator = pipeline(
    task=task,
    model=model,
    tokenizer=tokenizer,
    device=device_id,
    torch_dtype=torch.float16,
    **kwargs
)

Run with some text.

In [9]:
generator('アジャイルソフトウェア開発宣言の内容は')

[{'generated_text': 'アジャイルソフトウェア開発宣言の内容は、ソフトウェアの開発・運用に関する方針を簡潔にまとめたものです。\nこの文書には、「開発」や「生産」「販売」、「保守サービスなどの各業務プロセスにおいて『顧客満足』を追求していくこと」、そしてそれらを実現するための考え方として、『SMILE(Smile:笑顔で)』と書かれています。『信頼』『誠実さ'}]